# LightGBM - Parameter Tuning with Bayesian Optimization

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
import lightgbm as lgb

In [9]:
# train, test data set 불러오기
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
sample_submission = pd.read_csv("sample_submission.csv", index_col=0)

In [10]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [11]:
# oulier 제거
for column in train.columns[2:-1]:
    test_min_value = test[column].min()
    test_max_value = test[column].max()

    for column in train.columns[2:-1]:
        train = train.drop(index=train[(train[column] <= test_min_value -100) | (train[column] >= test_max_value +100)].index)

    train = train.reset_index(drop=True)
    
train.shape

(199880, 23)

In [12]:
# train set의 문제지(X), 정답지(y) 나누기
train_X = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

In [13]:
train_X.shape

(199880, 21)

In [14]:
# validation 진행하기 위한 train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, 
                                                    train_y, 
                                                    stratify = train_y,
                                                    test_size=0.2, 
                                                    random_state=1234)

In [14]:
# kfold = KFold(n_splits=6, random_state= 0, shuffle = True)
kfold_s = StratifiedKFold(n_splits=6, random_state= 0, shuffle = True)

# Bayesian Optimization

## 1. parameters to be tuned

In [18]:
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):

    params = {'application':'multiclass','num_iterations':1000, 'learning_rate':0.1, 'early_stopping_round':500, 'metric':'multi_logloss','num_class':19}
    params["num_leaves"] = round(num_leaves)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0) 
    params['max_depth'] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['multi_logloss'])
    return max(cv_result['multi_logloss'])

## 2. Set the range for each parameter

In [20]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (100,300),
                                        'feature_fraction': (0.1,0.3), # 열 샘플링 / default값인 1.0보다 작으면 각각의 tree를 training 시키기 전에 지정한 값만큼의 feature만 선택
                                        'bagging_fraction': (0.1,0.3), # 행 샘플링 / resampling 없이 data를 random으로 선택한다
                                        'max_depth': (1,9),
                                        'lambda_l1': (0.1,0.3), # L1 정규화 / 보통 default값인 0으로 놔둠
                                        'lambda_l2': (0.1,0.3), # L2 정규화 / 보통 default값인 0으로 놔둠
                                        'min_split_gain': (0.1,0.3),
                                        'min_child_weight':(0.1,0.3) }, random_state=1234)

In [21]:
# type_num에 1씩 더해서 type_num을 0부터가 아닌 1부터 시작하도록 만들어줌
ytrain_temp = y_train + 1

In [22]:
# 이건 그냥 확인하려고 쓴 코드
len(np.unique(y_train))

19

## 3. Put all together

In [23]:
def bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=20, n_folds=6, random_seed=1234, n_estimators=1000, learning_rate=0.1, output_process=False):
    # prepare data
    #train_data = lgb.Dataset(data=X, label=y, categorical_feature = categorical_feats, free_raw_data=False)
    
    # parameters
    train_data = lgb.Dataset(data=X, label=y,  free_raw_data=False)

    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
      # params = {'application':'multiclass','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'multi_logloss'}
        params = {'application':'multiclass','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':500, 'metric':'multi_logloss','num_class':19}

        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['multi_logloss'])
        #print(cv_result.keys())
        return max(cv_result['multi_logloss-mean'])
    
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (100,300),
                                            'feature_fraction': (0.1,0.3),
                                            'bagging_fraction': (0.1,0.3),
                                            'max_depth': (1,9),
                                            'lambda_l1': (0.1,0.3),
                                            'lambda_l2': (0.1,0.3),
                                            'min_split_gain': (0.1,0.3),
                                            'min_child_weight':(0.1,0.3) }, random_state=1234)
       
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result_200220_4.csv")
    
    # return best parameters
    return lgbBO.max["params"]

In [56]:
# opt_params.res

In [24]:
np.unique(ytrain_temp)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19], dtype=int64)

##  4. Best parameters 

In [24]:
# -----> input
opt_params = bayes_parameter_opt_lgb(X_train, y_train, 
                                     init_round=5, #5
                                     opt_round=20, # 10
                                     n_folds=6, # 3
                                     random_seed=1234, 
                                     n_estimators=1000,# 100 
                                     learning_rate=0.1) # 0.05

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.377191 + 0.00500139
[400]	cv_agg's multi_logloss: 0.372478 + 0.00491562
[600]	cv_agg's multi_logloss: 0.372464 + 0.00491044
[800]	cv_agg's multi_logloss: 0.372464 + 0.00491044
|  1        |  1.896    |  0.1383   |  0.2244   |  0.1875   |  0.2571   |  7.24     |  0.1545   |  0.1553   |  260.4    |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.381695 + 0.00465627
[400]	cv_agg's multi_logloss: 0.373207 + 0.00456913
[600]	cv_agg's multi_logloss: 0.373227 + 0.00457699
[800]	cv_agg's multi_logloss: 0.373227 + 0.00457699
|  2        |  1.911    |  0.2916   |  0.2752   |  0.1716   |  0.2002   |  6.468    |  0.2425   |  0.1741   |  212.2    |


...

[200]	cv_agg's multi_logloss: 0.722516 + 0.00455415
[400]	cv_agg's multi_logloss: 0.623732 + 0.0053525
[600]	cv_agg's multi_logloss: 0.582016 + 0.00577416
[800]	cv_agg's multi_logloss: 0.557829 + 0.00597671
[1000]	cv_agg's multi_logloss: 0.54195 + 0.00605628
|  23       |  2.198    |  0.1      |  0.1      |  0.2876   |  0.3      |  1.0      |  0.3      |  0.1      |  166.6    |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.722599 + 0.00459311
[400]	cv_agg's multi_logloss: 0.623688 + 0.00539957
[600]	cv_agg's multi_logloss: 0.581946 + 0.00579344
[800]	cv_agg's multi_logloss: 0.557795 + 0.00598482
[1000]	cv_agg's multi_logloss: 0.541895 + 0.00607738
|  24       |  2.198    |  0.1038   |  0.1109   |  0.2856   |  0.1131   |  1.009    |  0.2976   |  0.2536   |  258.3    |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.691816 + 0.00505177
[400]	cv_agg's multi_logloss: 0.605867 + 0.00549233
[600]	cv_agg's multi_logloss: 0.568905 + 0.00570778
[800]	cv_agg's multi_logloss: 0.547165 + 0.00575307
[1000]	cv_agg's multi_logloss: 0.532575 + 0.00578148
|  25       |  2.175    |  0.1      |  0.3      |  0.1      |  0.3      |  1.0      |  0.3      |  0.3      |  121.3    |


In [25]:
print(opt_params)

{'bagging_fraction': 0.1, 'feature_fraction': 0.1, 'lambda_l1': 0.3, 'lambda_l2': 0.3, 'max_depth': 1.0, 'min_child_weight': 0.1, 'min_split_gain': 0.1, 'num_leaves': 100.0}
